In [1]:
!tar zxvf data.tar.gz
!tar zxvf deepcomedy.tar.gz

data/
data/orlando.txt
data/divina_textonly.txt
data/divina.txt
data/divina_syll_textonly.txt
data/orlando-textonly.txt
data/divina_syll.txt
data/.ipynb_checkpoints/
data/.ipynb_checkpoints/orlando-checkpoint.txt
data/.ipynb_checkpoints/orlando-textonly-checkpoint.txt
data/.ipynb_checkpoints/divina_textonly-checkpoint.txt
data/.ipynb_checkpoints/divina_syll-checkpoint.txt
data/.ipynb_checkpoints/divina-checkpoint.txt
data/.ipynb_checkpoints/divina_syll_textonly-checkpoint.txt
deepcomedy/
deepcomedy/util/
deepcomedy/util/predicate.py
deepcomedy/util/__pycache__/
deepcomedy/util/__pycache__/predicate.cpython-37.pyc
deepcomedy/util/__pycache__/__init__.cpython-37.pyc
deepcomedy/util/__init__.py
deepcomedy/util/.ipynb_checkpoints/
deepcomedy/util/.ipynb_checkpoints/predicate-checkpoint.py
deepcomedy/models/
deepcomedy/models/layers.py
deepcomedy/models/transformer.py
deepcomedy/models/__pycache__/
deepcomedy/models/__pycache__/layers.cpython-37.pyc
deepcomedy/models/__pycache__/__init__.cp

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from deepcomedy.models.transformer import *
from deepcomedy.preprocessing import load_verses
import re
import tensorflow as tf
import numpy as np

In [4]:
corpus_path = "data/divina_syll_textonly.txt"
raw_corpus = open(corpus_path, "rb").read().decode(encoding="utf-8")

In [5]:
raw_corpus = raw_corpus.strip()

In [6]:
corpus = '\n'.join([line.strip()[1:] for line in raw_corpus.split('\n')])
corpus = re.sub(r'(.)', r'\1 ', corpus).strip()
corpus = re.sub(r'  ', ' <SEP> ', corpus)
corpus = re.sub(r'\n{2,}', ' <EOT> ', corpus)
corpus = re.sub(r'\n', ' <EOV> ', corpus)
corpus = re.sub(r' {2,}', ' ', corpus)
corpus = corpus + ' <EOT>'

In [24]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=False, filters='', lower=False)
tokenizer.fit_on_texts([corpus])
enc_corpus = tokenizer.texts_to_sequences([corpus])[0]
vocab_size = len(tokenizer.word_index) + 1

In [32]:

len(enc_corpus)

671248

In [44]:
window_size = 100
corpus_windows = []

for i in range(0, len(enc_corpus) - window_size, window_size):
    corpus_windows.append(enc_corpus[i:i+window_size])

corpus_windows = tf.convert_to_tensor(corpus_windows)

In [48]:
BUFFER_SIZE = len(corpus_windows)
BATCH_SIZE = 64
steps_per_epoch = len(corpus_windows) // BATCH_SIZE

max_length_targ = corpus_windows.shape[1]

dataset = tf.data.Dataset.from_tensor_slices(corpus_windows).shuffle(
    BUFFER_SIZE
)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [134]:
epochs = 10
num_layers = 12
d_model = 256
num_heads = 8
dff = 1024

In [137]:
decoder = DecoderOnlyModel(num_layers, d_model, num_heads, dff, vocab_size, 1000)
decoderTrainer = DecoderOnlyTrainer(decoder, checkpoint_save_path='checkpoints/decoder-only-12')

In [147]:
decoderTrainer.train(dataset, epochs * 2)

Epoch 1 Batch 0 Loss 0.9504 Accuracy 0.6711
Epoch 1 Batch 50 Loss 0.9473 Accuracy 0.6760
Epoch 1 Batch 100 Loss 0.9609 Accuracy 0.6713
Epoch 1 Loss 0.9613 Accuracy 0.6712
Time taken for 1 epoch: 21.88 secs

Epoch 2 Batch 0 Loss 0.9208 Accuracy 0.6837
Epoch 2 Batch 50 Loss 0.9352 Accuracy 0.6799
Epoch 2 Batch 100 Loss 0.9489 Accuracy 0.6755
Epoch 2 Loss 0.9493 Accuracy 0.6754
Time taken for 1 epoch: 22.00 secs

Epoch 3 Batch 0 Loss 0.9309 Accuracy 0.6832
Epoch 3 Batch 50 Loss 0.9241 Accuracy 0.6824
Epoch 3 Batch 100 Loss 0.9366 Accuracy 0.6786
Epoch 3 Loss 0.9375 Accuracy 0.6784
Time taken for 1 epoch: 21.23 secs

Epoch 4 Batch 0 Loss 0.9070 Accuracy 0.6888
Epoch 4 Batch 50 Loss 0.9135 Accuracy 0.6866
Epoch 4 Batch 100 Loss 0.9258 Accuracy 0.6823
Epoch 4 Loss 0.9265 Accuracy 0.6820
Time taken for 1 epoch: 21.27 secs

Epoch 5 Batch 0 Loss 0.8753 Accuracy 0.6955
Epoch 5 Batch 50 Loss 0.9018 Accuracy 0.6901
Epoch 5 Batch 100 Loss 0.9146 Accuracy 0.6860
Saving checkpoint for epoch 5 at chec

In [140]:
def generate_greedy(input):
    input = tf.expand_dims(input, 0)
    result = ""

    for i in range(400):
        _, combined_mask, _ = create_masks(
            input, input
        )

        # predictions.shape == (batch_size, seq_len, vocab_size)
        predictions = decoder(
            input,
            False,
            combined_mask
        )
        # select the last character from the seq_len dimension
        predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)

        predicted_id = tf.random.categorical(tf.reshape(predictions, (-1, vocab_size)), 1)

        # concatenate the predicted_id to the output which is given to the decoder as its input.
        input = tf.concat(
            [tf.cast(input, dtype=tf.int32), tf.cast(predicted_id, dtype=tf.int32)],
            axis=-1,
        )
        
        result += tokenizer.index_word[predicted_id.numpy()[0][0]] + " "

        # return the result if the predicted_id is equal to the end token
        if predicted_id == tokenizer.word_index["<EOT>"]:
            return result
    return result

In [149]:
input = tokenizer.texts_to_sequences(['<EOV>'])

In [148]:
generate_greedy(input[0])

'c o m ’ <SEP> | i o <SEP> | v i | d i , <SEP> i l <SEP> | t u o <SEP> | v a | l o r <SEP> | m i <SEP> | r i | s p o n | d o , <EOV> c h e <SEP> | s e m | p r e <SEP> | v a | l o | r e <SEP> a l <SEP> | c a | p o <SEP> | s e | d e r | n o . <EOT> '